<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/TP1_M6_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de Threads en Java

## Creacion de archivos

In [ ]:
%%writefile App.java
import java.util.ArrayList;
import java.util.List;
import java.util.Map;
import java.util.Random;

public class App {
  public static final int MIN_VALUE = -32;
  public static final int MAX_VALUE = 32;
  public static final int NUMBER_OF_MATRICES = 2;

  public static void main(String[] args) throws Exception
  {
    // Inicializamos matrices
    int matrixSize = parseInteger(args);
    List<int[][]> matrices = initializeMatrices(NUMBER_OF_MATRICES, matrixSize, MIN_VALUE, MAX_VALUE);
    int[][] matrixA = matrices.get(0);
    int[][] matrixB = matrices.get(1);

    // Creamos la caluladora
    MatrixCalculatorFactory calculatorFactory = new MatrixCalculatorFactory();
    MatrixCalculator sequentialMatrixCalculator = calculatorFactory.createMatrixCalculator(MatrixCalculatorFactory.SEQUENTIAL_CALCULATOR);
    MatrixCalculator parallelMatrixCalculator = calculatorFactory.createMatrixCalculator((MatrixCalculatorFactory.PARALLEL_CALCULATOR));
    
    // Calculamos resultados
    int[][] sequentialResult = sequentialMatrixCalculator.multiply(matrixA, matrixB);
    int[][] parallelResult = parallelMatrixCalculator.multiply(matrixA, matrixB);

    // Mostramos por pantalla los resultados o el error si hubiera.
    if (compare(sequentialResult, parallelResult))
    {
      List<Map.Entry<String, int[][]>> labeledMatrices = List.of(
        Map.entry("Matriz A", matrixA),
        Map.entry("Matriz B", matrixB),
        Map.entry("Resultado secuencial", sequentialResult),
        Map.entry("Resultado paralelo", parallelResult)
      );
      printMatrices(labeledMatrices);
    } 
    else
    {
      System.out.println("Las dos matrices no dieron igual!!");
    }
  }
  
  public static int parseInteger(String args[])
  {
    int squareMatrixSize = Integer.MIN_VALUE;
    try
    {
      squareMatrixSize = Integer.parseInt(args[0]);
      if(squareMatrixSize < 5 || squareMatrixSize > 20) 
      {
        System.err.println("El tamaño de la matriz debe ser un número entero entre 5 y 20");
        System.exit(1);
      }
    }
    catch (NumberFormatException e)
    {
      System.err.println("El primer parámetro debe ser un número entero");
      System.exit(1);
    }
    return squareMatrixSize;
  }
  
  // Inicializa una matrix cuadrada de tamaño n con números aleatorios entre min (inclusivo) y max (inclusivo)
  public static int[][] initializeMatrix(int n, int min, int max)
  {
    int[][] matrix = new int[n][n];
    Random randomizer = new Random();
    for(int i = 0; i < n; i++)
    {
      for(int j = 0; j < n; j++)
      {
        matrix[i][j] = randomizer.nextInt((max + 1) - min) + min;
      }
    }
    return matrix;
  }

  // Inicializa una lista de matrices cuadradas de tamaño n con números aleatorios entre min (inclusivo) y max (inclusivo)
  public static List<int[][]> initializeMatrices(int numberOfMatrices, int matrixDimension, int minValue, int maxValue)
  {
    List<int[][]> matrices = new ArrayList<>();
    for(int i = 0; i < numberOfMatrices; i++)
    {
      matrices.add(initializeMatrix(matrixDimension, minValue, maxValue));
    }
    return matrices;
  }
  

  // Compara dos matrices y devuelve true si son iguales, false en caso de que cualquier elemento de igual indice sea distinto
  public static boolean compare(int[][] matrixA, int[][] matrixB)
  {
    for(int i = 0; i < matrixA.length; i++)
    {
      for(int j = 0; j < matrixA.length; j++)
      {
        if(matrixA[i][j] != matrixB[i][j])
        {
          return false;
        }
      }
    }
    return true;
  }
  
  // Recibe una matriz y la imprime por consola
  public static void printMatrix(int[][] matrix)
  {
    for(int i = 0; i < matrix.length; i++)
    {
      System.out.print("|");
      for(int j = 0; j < matrix.length; j++)
      {
        System.out.print(String.format("%5d|",matrix[i][j]));
      }
      System.out.println();
    }
    System.out.println();
  }

    // Recibe una lista de tuplas de nombre, matriz y las imprime por consola
  public static void printMatrices(List<Map.Entry<String, int[][]>> matrices)
  {
    for(Map.Entry<String, int[][]> entryMap : matrices)
    {
      System.out.println(entryMap.getKey());
      printMatrix(entryMap.getValue());
    }
  }
}


Overwriting App.java


ConcurrentMatrixMultiplicator es una estrategia de multiplicacion que consiste en crear una pool de tantos Threads como filas tengan las matrices cuadradas y luego iniciarlos todos de manera que iteren por las columnas de sus filas para hacer la multiplicación con la otra matriz.

In [ ]:
%%writefile ConcurrentMatrixMultiplicator.java
import java.util.ArrayList;
import java.util.List;

public class ConcurrentMatrixMultiplicator implements IMatrixMultiplicationStrategy
  {
    public int[][] matrixMultiplication(int[][] a, int[][] b) {
      int rowsA = a.length;
      int rowsB = b.length;
      int[][] ch = new int[rowsA][rowsB];
      
      // LLevamos registro de todos los threads lanzados para esperarlos antes de devolver el resultado
      List<Thread> threadPool = new ArrayList<Thread>(a.length*b.length);
      for (int i = 0; i < a.length; i++) {
        // Las variables en el Thread tienen que ser inmutables.
        final int iCopy = i;
        Thread t = new Thread(() -> {
          for (int j = 0; j < b[0].length; j++) {
            for (int k = 0; k < a[0].length; k++) {
              ch[iCopy][j] += a[iCopy][k] * b[k][j];
  
            }
          }
        });

        // Agregamos el thread a la pool para lanzarlo luego.
        // Si bien podriamos lanzar el hilo recien creado ahora mismo, testeos locales con N > 1000 tuvieron mejor rendimiento
        //  al crear la pool con todos los hilos y lanzarlos luego.
        threadPool.add(t);
      }

      // Iniciamos los hilos en la pool.
      threadPool.forEach(t -> t.start());

      // Esperamos los hilos que todavía estén procesando la multiplicación
      threadPool.forEach(t -> {
        try 
        {
          t.join();
        } catch (InterruptedException e)
        {
          System.out.println("Ocurrio un error durante la multiplicacion en paralelo");
          e.printStackTrace();
        }
      });

      return ch;
    }
  }

Writing ConcurrentMatrixMultiplicator.java


In [ ]:
%%writefile IMatrixMultiplicationStrategy.java
public interface IMatrixMultiplicationStrategy
{
  // Returns product of two matrices
  public int[][] matrixMultiplication(int[][] a, int[][] b);
}

Writing IMatrixMultiplicationStrategy.java


Calculadora de matrices que utiliza una estrategia para hacer los calculos

In [ ]:
%%writefile MatrixCalculator.java
public class MatrixCalculator
{
  private IMatrixMultiplicationStrategy multiplicationStrategy_;

  public MatrixCalculator(IMatrixMultiplicationStrategy multiplicationStrategy) 
  {
    this.multiplicationStrategy_ = multiplicationStrategy;
  }

  public int[][] multiply(int[][] a, int[][] b)
  {
    return this.multiplicationStrategy_.matrixMultiplication(a, b);
  }
}

Writing MatrixCalculator.java


Factory de calculadora de matrices que setea las estrategias según se indique en el parametro

In [ ]:
%%writefile MatrixCalculatorFactory.java
public class MatrixCalculatorFactory
  {
    public static final String SEQUENTIAL_CALCULATOR = "sequential";
    public static final String PARALLEL_CALCULATOR = "parallel"; 

    public MatrixCalculator createMatrixCalculator(String strategy)
    {
      MatrixCalculator calculator = null;
      switch (strategy) {
        case SEQUENTIAL_CALCULATOR:
          calculator = new MatrixCalculator(new SequentialMatrixMultiplicator());
          break;
        case PARALLEL_CALCULATOR:
          calculator = new MatrixCalculator(new ConcurrentMatrixMultiplicator());
          break;
        default:
          System.out.println("Por favor, ingrese un algoritmo de multiplicacion valido");
          break;
      }
      return calculator;
    }
}

Writing MatrixCalculatorFactory.java


SequentialMatrixMultiplicator es una estrategia de multiplicación que consiste en iterar secuencialmente por todos los elementos de una matriz y hacer la multiplicación con la otra

In [ ]:
%%writefile SequentialMatrixMultiplicator.java
public class SequentialMatrixMultiplicator implements IMatrixMultiplicationStrategy
  {
    public int[][] matrixMultiplication(int[][] a, int[][] b) {
      //Multiply matrix a by matrix b
      int rowsA = a.length;
      int rowsB = b.length;
      int[][] cs = new int[rowsA][rowsB];
      
      for (int i = 0; i < a.length; i++) {
        for (int j = 0; j < b[0].length; j++) {
          for (int k = 0; k < a[0].length; k++) {
            cs[i][j] += a[i][k] * b[k][j];
          }
        }
      }
      return cs;
    }
  }

Writing SequentialMatrixMultiplicator.java


## Compilacion

In [ ]:
!javac *.java

## Ejecucion

In [ ]:
!java App 20

Matrix A
|   12|   -4|   -2|   -8|   19|   -5|   23|   32|  -26|   -3|   -2|   23|   27|    0|    8|   27|  -28|   -8|    4|  -30|
|  -24|  -13|    0|   11|   -4|    2|   31|  -13|   22|   18|   -9|   19|  -10|   22|   26|   20|    1|   -5|   -9|  -26|
|   12|  -16|   13|  -14|  -24|   23|   19|   27|  -22|   10|   -4|  -10|   -7|   10|  -10|    3|  -25|   -6|    9|    9|
|  -17|    6|    5|  -13|  -27|   -3|  -14|   17|  -25|   13|  -20|    8|   19|   26|   20|  -31|   18|   15|  -14|   29|
|   30|    0|  -30|  -32|   25|   24|    0|  -11|  -30|  -20|    6|   20|   15|    3|   -7|   32|   20|    8|   21|  -11|
|  -15|   31|  -27|    2|   11|   27|  -30|   -2|    8|    3|    6|   -3|    5|   -2|  -24|  -16|    9|    9|  -28|  -18|
|  -18|  -22|   17|  -28|  -12|  -21|  -17|    7|   23|   11|  -14|   27|   21|   25|   14|   -6|  -27|   26|   31|  -14|
|   -2|   23|    0|  -25|   10|   29|  -27|   23|   11|    3|  -12|    8|   -3|  -23|   24|  -22|   12|   -3|   -7|   18|
|    7|   -1|  

# Ejemplo threads en Python

In [ ]:
%%writefile threads.py
import random
import sys
import threading
import os

def generate_random_matrix(N, min_value, max_value):
    matrix = []
    for i in range(N):
        row = []
        for j in range(N):
            row.append(random.randint(min_value, max_value))
        matrix.append(row)
    return matrix

def matrix_multiplication(matrixA, matrixB):
    result = []
    for i in range(len(matrixA)):
        row = []
        for j in range(len(matrixB[0])):
            sum = 0
            for k in range(len(matrixB)):
                sum += matrixA[i][k] * matrixB[k][j]
            row.append(sum)
        result.append(row)
    return result

def concurrent_matrix_multiplication(matrixA, matrixB):
    result = []
    threads = []
    for i in range(len(matrixA)):
        row = []
        # row va a contener la fila i de la matriz resultante. La inicializamos con ceros
        for j in range(len(matrixB[0])):
            row.append(0)
        result.append(row)

        # Creamos tantos hilos como filas tenga la matriz A. Estos hilos calcularan el producto entre la fila i de la matriz A y la matriz B
        t = threading.Thread(target=calculate_row, args=(matrixA, matrixB, result, i))
        t.start()
        threads.append(t)
    for t in threads:
        # Esperamos a los hilos que todavia esten calculando antes de devolver el resultado
        t.join()
    return result

def calculate_row(matrixA, matrixB, result, row_index):
    for j in range(len(matrixB[0])):
        sum = 0
        for k in range(len(matrixB)):
            sum += matrixA[row_index][k] * matrixB[k][j]
        result[row_index][j] = sum

def print_matrix(matrix):
    for row in matrix:
        print(row)

def parse_parameters():
    if len(sys.argv) != 2:
        print(f"Uso: python {os.path.basename(__file__)} N")
        sys.exit(1)
    N = int(sys.argv[1])
    if N < 5 or N > 20:
        print("N debe estar entre 5 y 20")
        sys.exit(1)
    return N

MIN_VALUE = -32
MAX_VALUE = 32
if __name__ == "__main__":
    N = parse_parameters()

    matrixA = generate_random_matrix(N, MIN_VALUE, MAX_VALUE)
    matrixB = generate_random_matrix(N, MIN_VALUE, MAX_VALUE)

    sequential_result = matrix_multiplication(matrixA, matrixB)
    concurrent_result = concurrent_matrix_multiplication(matrixA, matrixB)

    if sequential_result == concurrent_result:
        print("Matriz A:")
        print_matrix(matrixA)
        print("Matriz B:")
        print_matrix(matrixB)
        print("Resultado secuencial:")
        print_matrix(sequential_result)
        print("Resultado concurrente:")
        print_matrix(parallel_result)
    else:
        print("Las dos matrices no dieron igual!!")

Overwriting threads.py


In [ ]:
!python threads.py 6

Matriz A:
[14, -12, 3, 14, -1, -7]
[16, 2, -5, -27, -12, -20]
[-9, 32, -5, 23, -11, -20]
[-10, 29, -23, 10, -19, -28]
[-17, 0, 25, -29, -7, -32]
[-31, 25, -25, 0, -15, -31]
Matriz B:
[-9, -22, -21, -25, -3, 19]
[-16, 6, 6, 11, 30, 9]
[24, -4, -24, -32, -27, 30]
[16, -6, -11, -27, -29, -4]
[-32, 2, 1, -21, 25, 2]
[-25, -8, -29, -23, -19, 25]
Resultado secuencial:
[569, -422, -390, -774, -781, 15]
[156, -22, 661, 1223, 1010, -244]
[669, 410, 817, 807, 560, -647]
[542, 612, 1619, 2078, 1288, -1397]
[1313, 690, 997, 1291, 650, -271]
[534, 1150, 2285, 2878, 1732, -1919]
Resultado concurrente:
Traceback (most recent call last):
  File "/content/threads.py", line 86, in <module>
    print_matrix(parallel_result)
NameError: name 'parallel_result' is not defined


#Ejemplo threads en C++

In [ ]:
%%writefile threads.cpp
#include <iostream>
#include <vector>
#include <random>
#include <thread>

std::vector<std::vector<int>> generate_random_matrix(int N, int min_value, int max_value) 
{
    //Generamos un numero realmente aleatorio con el hardware del dispositivo
    std::vector<std::vector<int>> matrix(N, std::vector<int>(N));
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dis(min_value, max_value);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            matrix[i][j] = dis(gen);
        }
    }
    return matrix;
}

std::vector<std::vector<int>> matrix_multiplication(const std::vector<std::vector<int>>& matrixA, const std::vector<std::vector<int>>& matrixB) 
{
    //Algoritmo estandar para multiplicar secuencialmente dos matrices
    int N = matrixA.size();
    std::vector<std::vector<int>> result(N, std::vector<int>(N));
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int sum = 0;
            for (int k = 0; k < N; k++) {
                sum += matrixA[i][k] * matrixB[k][j];
            }
            result[i][j] = sum;
        }
    }
    return result;
}

std::vector<std::vector<int>> concurrent_matrix_multiplication(const std::vector<std::vector<int>>& matrixA, const std::vector<std::vector<int>>& matrixB) 
{
    //Inicializamos las variables
    int N = matrixA.size();
    std::vector<std::thread> threads;
    std::vector<std::vector<int>> result(N, std::vector<int>(N));

    //Guardamos una funcion anonima en calculate_row que puede acceder a N y recibe un row_index.
    // Esta funcion sera usada en los threads, y acceder a N es seguro en este contexto puesto que es solo lectura.
    auto calculate_row = [&](int row_index) {
        for (int j = 0; j < N; j++) {
            int sum = 0;
            for (int k = 0; k < N; k++) {
                sum += matrixA[row_index][k] * matrixB[k][j];
            }
            result[row_index][j] = sum;
        }
    };

    for (int i = 0; i < N; i++) {
        //Construimos el objeto y lo insertamos al final del vector. El hilo inicia inmediatamente despues de creado el objeto
        threads.emplace_back(calculate_row, i);
    }

    //Esperamos a que todos los hilos terminen antes de devolver el resultado
    for (auto& t : threads) {
        t.join();
    }
    return result;
}

void print_matrix(const std::vector<std::vector<int>>& matrix) 
{
    for (const auto& row : matrix) {
        std::cout << "| ";
        for (int x : row) {
            std::cout << x << "| ";
        }
        std::cout << '\n';
    }
}

int parse_arguments(int argc, char* argv[]) 
{
    if (argc != 2) {
        std::cerr << "Uso: " << argv[0] << " N\n";
        return 1;
    }
    int N = std::stoi(argv[1]);
    if (N < 5 || N > 20) {
        std::cerr << "N debe estar entre 5 y 20\n";
        return 1;
    }
    return N;
}

int main(int argc, char* argv[]) 
{
    int N = parse_arguments(argc, argv);

    constexpr int MIN_VALUE = -32;
    constexpr int MAX_VALUE = 32;

    auto matrixA = generate_random_matrix(N, MIN_VALUE, MAX_VALUE);
    auto matrixB = generate_random_matrix(N, MIN_VALUE, MAX_VALUE);

    auto sequential_result = matrix_multiplication(matrixA, matrixB);
    auto concurrent_result = concurrent_matrix_multiplication(matrixA, matrixB);

    if (sequential_result == concurrent_result) {
        std::cout << "Matrix A:\n";
        print_matrix(matrixA);
        std::cout << "Matrix B:\n";
        print_matrix(matrixB);
        std::cout << "Resultado secuencial:\n";
        print_matrix(sequential_result);
        std::cout << "Resultado concurrente:\n";
        print_matrix(concurrent_result);
    } else {
        std::cout << "Las dos matrices no dieron igual!!\n";
    }
}

Overwriting threads.cpp


In [ ]:
!g++ -std=c++11 -pthread threads.cpp -o program

In [ ]:
!./program 5

Matrix A:
| 7| -18| -24| -14| 8| 
| -25| 22| -23| 16| 14| 
| -16| 16| 13| -10| 19| 
| 14| 11| 22| 3| 12| 
| 0| 14| 8| 25| 2| 
Matrix B:
| 28| 7| -16| -24| 1| 
| 29| -8| 22| 24| -25| 
| -17| -7| 8| -15| -21| 
| 16| -1| -31| 25| 26| 
| 15| -21| 31| -1| -12| 
Resultado secuencial:
| -22| 207| -18| -598| 501| 
| 795| -500| 638| 1859| 156| 
| -80| -720| 1611| 304| -1177| 
| 565| -399| 473| -339| -789| 
| 700| -235| -341| 839| 108| 
Resultado concurrente:
| -22| 207| -18| -598| 501| 
| 795| -500| 638| 1859| 156| 
| -80| -720| 1611| 304| -1177| 
| 565| -399| 473| -339| -789| 
| 700| -235| -341| 839| 108| 
